# The Coffee Cart Revenue Mystery

## Story

You’ve been hired by a small chain of mobile coffee carts that operate across a few busy city locations.
The owner suspects that some carts are under-reporting revenue—not maliciously, but due to messy logging habits.
Every cart records its sales in a simple export: product, quantity, and timestamp, but the system didn’t enforce consistent data entry, so now they need someone to make sense of it.

You receive a week of raw entries. Your job is to clean the inconsistencies, summarise patterns, and help the owner understand how each cart actually performed.

## Tasks

### Task 1 — Clean the basic fields
- Standardise the `product` names (capitalisation/spacing).
- Convert `qty` values into valid integers.
- For clearly unusable rows (e.g., missing product *and* invalid quantity), either:
  - set the entire row to `NaN`, or  
  - drop the row.  
  Explain which approach you chose and why.

---

### Task 2 — Compute revenues
- Create a new column `revenue` as `qty * price`.
- Ensure there are no invalid or negative `revenue` values.
- If `qty` is missing but the row is otherwise valid, set `revenue` to 0.

---

### Task 3 — Summaries for management
Create a cart-level summary table showing, for each `cart`:
- total items sold,
- total revenue,
- number of problematic rows detected (e.g., rows with invalid or corrected quantities).

Sort carts from highest to lowest total revenue.

---

### Task 4 — Detect suspicious behavior
Using only the cleaned data:
- Check for inconsistent pricing per product (e.g., `Latte` should always cost 5.0).
- For each `cart` × `product` combination, compare the mean observed price to the expected price.
- Flag combinations where the mean price deviates by more than 5% from the expected price.


## Data

In [1]:
import pandas as pd
import numpy as np

data = pd.DataFrame([
    {"cart": "North",  "product": "Latte",      "qty": "2",   "price": 5.0, "timestamp": "2025-01-11 08:23"},
    {"cart": "North",  "product": "latte ",     "qty": " 1",  "price": 5.0, "timestamp": "2025-01-11 09:10"},
    {"cart": "East",   "product": "Espresso",   "qty": "3",   "price": 3.5, "timestamp": "2025-01-11 07:40"},
    {"cart": "East",   "product": "espresso",   "qty": "one", "price": 3.5, "timestamp": "2025-01-11 10:05"},
    {"cart": "West",   "product": "Mocha",      "qty": 2,     "price": 6.0, "timestamp": "2025-01-11 11:15"},
    {"cart": "West",   "product": "MOCHA",      "qty": "2 ",  "price": 6.0, "timestamp": "2025-01-11 12:40"},
    {"cart": "North",  "product": None,         "qty": "2",   "price": 5.0, "timestamp": "2025-01-11 13:00"},
    {"cart": "East",   "product": "Latte",      "qty": None,  "price": 5.0, "timestamp": "2025-01-11 14:20"},
])


## Task 1 — Clean the basic fields
- Standardise the `product` names (capitalisation/spacing).
- Convert `qty` values into valid integers.
- For clearly unusable rows (e.g., missing product *and* invalid quantity), either:
  - set the entire row to `NaN`, or  
  - drop the row.  
  Explain which approach you chose and why.

In [ ]:
def standardise_data(df):
    df = df.copy()

    # keep raw columns
    df['product_raw'] = df['product']
    df['qty_raw'] = df['qty']

    # product
    def plain_text(text):
        if pd.isna(text):
            return None
        return text.lower().strip()

    df['product'] = df['product'].apply(plain_text)

    # qty
    def qty_fix(val):
        word_to_int = {
            "zero": 0, "one": 1, "two": 2, "three": 3,
            "four": 4, "five": 5,
            "0": 0, "1": 1, "2": 2, "3": 3, "4": 4, "5": 5
        }
        if isinstance(val, int):
            return val
        try:
            return int(val)
        except:
            pass
        if isinstance(val, str):
            cleaned = plain_text(val)
            if cleaned in word_to_int:
                return word_to_int[cleaned]
        return np.nan   # use np.nan not None

    df['qty'] = df['qty'].apply(qty_fix)

    # define "problematic" rows.
    # e.g.: product was missing OR qty was missing OR qty got turned into NaN
    df['problematic'] = (
        df['product_raw'].isna() |
        df['qty_raw'].isna() |
        df['qty'].isna()
    )

    return df


In [3]:
df_clean = standardise_data(data)
df_clean

,cart,product,qty,price,timestamp,product_raw,qty_raw,problematic
0,North,latte,2.0,5.0,2025-01-11 08:23,Latte,2,False
1,North,latte,1.0,5.0,2025-01-11 09:10,latte,1,False
2,East,espresso,3.0,3.5,2025-01-11 07:40,Espresso,3,False
3,East,espresso,1.0,3.5,2025-01-11 10:05,espresso,one,False
4,West,mocha,2.0,6.0,2025-01-11 11:15,Mocha,2,False
5,West,mocha,2.0,6.0,2025-01-11 12:40,MOCHA,2,False
6,North,None,2.0,5.0,2025-01-11 13:00,None,2,True
7,East,latte,NaN,5.0,2025-01-11 14:20,Latte,None,True


## Task 2 — Compute revenues
- Create a new column `revenue` as `qty * price`.
- Ensure there are no invalid or negative `revenue` values.
- If `qty` is missing but the row is otherwise valid, set `revenue` to 0.

In [52]:
# Convert any None to np.nan BEFORE computing revenue
df_clean['qty'] = df_clean['qty'].astype('float')   # forces None → nan

# Now revenue works cleanly
df_clean['revenue'] = df_clean['qty'] * df_clean['price']

# Missing qty → revenue = 0
df_clean['revenue'] = df_clean['revenue'].fillna(0)


## Task 3 — Summaries for management
Create a cart-level summary table showing, for each `cart`:
- total items sold,
- total revenue,
- number of problematic rows detected (e.g., rows with invalid or corrected quantities).

Sort carts from highest to lowest total revenue.

In [53]:
summary  = df_clean.groupby(by = 'cart').agg(total_item_sold = ('qty', 'sum'),
                                            total_revenue = ('revenue', 'sum'),
                                            problematic = ('problematic', 'sum')).sort_values('total_revenue')

print('Summary:')
print('--------')
print(summary)

Summary:
--------
       total_item_sold  total_revenue  problematic
cart                                              
East               4.0           14.0            1
West               4.0           24.0            0
North              5.0           25.0            1


## Task 4 — Detect suspicious behavior
Using only the cleaned data:
- Check for inconsistent pricing per product (e.g., `Latte` should always cost 5.0).
- For each `cart` × `product` combination, compare the mean observed price to the expected price.
- Flag combinations where the mean price deviates by more than 5% from the expected price.


In [ ]:
expected_price = {
    'latte':5.0,
    'espresso':3.5,
    'mocha':6
}

expected_product = {
    5.0:'latte',
    3.5:'espresso',
    6:'mocha'
}

# expected product (fill NA)

df_clean['expected_product'] = df_clean['price'].map(expected_product)

# expected price (from expected product)
df_clean['expected_price'] = df_clean['expected_product'].map(expected_price)

# price diff
df_clean['price_diff'] = df_clean['price'] - df_clean['expected_price']

# deviation
df_clean['deviation'] = (
    (df_clean['price'] - df_clean['expected_price']).abs()
    / df_clean['expected_price']
)

# flag suspicious (>5%)

df_clean['flag'] = df_clean['deviation'].apply(
    lambda x: 'suspicious' if pd.notna(x) and x > 0.05 else 'ok'
)

df_clean.head()

In [70]:
summary = (
    df_clean
      .groupby('cart')
      .agg(
          total_items_sold      = ('qty', 'sum'),
          total_revenue         = ('revenue', 'sum'),
          problematic_rows      = ('problematic', 'sum'),   # from Task 1
          suspicious_price_rows = ('flag', lambda x: (x == 'suspicious').sum())
      )
      .sort_values('total_revenue', ascending=False)
)

print("Cart Performance Summary")
print("------------------------")
print(summary)


Cart Performance Summary
------------------------
       total_items_sold  total_revenue  problematic_rows  \
cart                                                       
North               5.0           25.0                 1   
West                4.0           24.0                 0   
East                4.0           14.0                 1   

       suspicious_price_rows  
cart                          
North                      0  
West                       0  
East                       0  


# === End of Challenge ===